In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# 워킹데이 리스트 불러오기
date_df = pd.read_csv('../../../Stock_Data/Data/stock_df/date_list.csv')
del date_df['Unnamed: 0']
del date_df['article']
date_df = date_df[date_df['Date'] <= '2019-12-31']

In [27]:
date_df

,Date,Category
0,2014-01-02,2
1,2014-01-02,3
2,2014-01-02,4
3,2014-01-02,5
4,2014-01-02,6
...,...,...
14695,2019-12-30,7
14696,2019-12-30,8
14697,2019-12-30,9
14698,2019-12-30,10


## 휴일 날짜 변경, 날짜, 카테고리별 Summary, 카테고리 원핫인코딩

#### 1) for문 사용해서 전체 종목 돌릴때

In [45]:
kospi20=['SK텔레콤','LG전자','이마트','기아차','SK하이닉스','한국전력',\
'현대백화점','셀트리온','KB금융','LG화학','아모레퍼시픽','기업은행','현대모비스','대우조선해양','GS건설',\
'S-Oil','CJ제일제당', '삼성전자','현대차','대한항공']

for index, kospi in enumerate(kospi20):
    #print(index)
    num = index+1
    article_df = pd.read_csv('C:/py/stock_trading_agent/article_df/selected20/{}_df.csv'.format(kospi))
    del article_df['Unnamed: 0']
    article_df = article_df.drop_duplicates('Title',keep = 'first')
    article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
    article_df = article_df.dropna(axis=0)
    article_df = article_df.reset_index(drop=True)
    #print(len(article_df))
    for i in range(len(article_df)):
        #print(i)
        article_df['Date'][i] = article_df['Date'][i].replace('.','-')
    article_df = article_df.sort_values('Date')

    
    while True:
        index = article_df['Date'].isin(date_df['Date']) == False
        holidays = article_df['Date'][index]
        #print(len(holidays))
        if len(holidays) == 0:
            break
        else:
            for i in range(len(holidays)):
                holiday = holidays.iloc[i]
                conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
                not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
                holidays.iloc[i] = not_holiday
            article_df['Date'][index] = holidays 
    
    merge_df = pd.merge(date_df, article_df, left_on=['Date','Category'], right_on=['Date','Category'], how='left')
    
    # one-hot encoding
    merge_df['politics']=0
    merge_df['economics']=0
    merge_df['IT']=0
    merge_df['finance']=0
    merge_df['realEstate']=0
    merge_df['international']=0
    merge_df['social']=0
    merge_df['life']=0
    merge_df['golf']=0
    merge_df['ent']=0

    for i in range(len(merge_df)):
        if merge_df['Category'][i] == 2:
            merge_df['politics'][i] = 1
        elif merge_df['Category'][i] == 3:
            merge_df['economics'][i] = 1
        elif merge_df['Category'][i] == 4:
            merge_df['IT'][i] = 1
        elif merge_df['Category'][i] == 5:
            merge_df['finance'][i] = 1
        elif merge_df['Category'][i] == 6:
            merge_df['realEstate'][i] = 1
        elif merge_df['Category'][i] == 7:
            merge_df['international'][i] = 1
        elif merge_df['Category'][i] == 8:
            merge_df['social'][i] = 1
        elif merge_df['Category'][i] == 9:
            merge_df['life'][i] = 1
        elif merge_df['Category'][i] == 10:
            merge_df['golf'][i] = 1
        elif merge_df['Category'][i] == 11:
            merge_df['ent'][i] = 1
        #print(i)

    del merge_df['Category']    
    merge_df.to_csv('C:/py/stock_trading_agent/article_df/selected20/{index}. {name}_df_V2.csv'.format(index=num,name=kospi), index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/py/stock_trading_agent/article_df/selected20/SK텔레콤_df.csv'

#### 2) 한 종목만 변경 - label categorize (1,0,-1)

In [29]:
article_df = pd.read_csv('../../../Sentiment Analysis/Code/handmade_sentitest/trainingset/CJ제일제당_traintestdata_3300.csv')
del article_df['Unnamed: 0']
article_df = article_df.drop_duplicates('Title',keep = 'first')
article_df = article_df.drop(article_df[article_df['Date'] == '2019.12.31'].index)
article_df = article_df.dropna(axis=0)
article_df = article_df.reset_index(drop=True)

# Date 형식 변경 및 sorting
for i in range(len(article_df)):
    article_df['Date'][i] = article_df['Date'][i].replace('.','-')
article_df = article_df.sort_values('Date')

# date_df에 없는 날짜 그 다음 날짜로 변경
while True:
    index = article_df['Date'].isin(date_df['Date']) == False
    holidays = article_df['Date'][index]
    #print(len(holidays))
    if len(holidays) == 0:
        break
    else:
        for i in range(len(holidays)):
            holiday = holidays.iloc[i]
            conv_hd = datetime.datetime.strptime(holiday,'%Y-%m-%d')
            not_holiday = str(conv_hd + datetime.timedelta(days=1))[:10]
            holidays.iloc[i] = not_holiday
        article_df['Date'][index] = holidays 

# cj기사에 이상치 변경
article_df.loc[article_df['label']=='-'] = -1

# 긍정, 부정, 중립 원핫인코딩
article_df['pos']=0
article_df['neu']=0
article_df['neg']=0

for i in range(len(article_df)):
    if article_df.iloc[i]['label'] == '1':
        article_df['pos'].iloc[i] = 1
        #print(article_df.iloc[i]['pos'])
    elif article_df.iloc[i]['label'] == '0':
        article_df['neu'].iloc[i] = 1
    else:
        article_df['neg'].iloc[i] = 1
#     print(i)

article_df['count'] = 1
        
# article_df 날짜, 카테고리별 grouping
article_df2 = article_df[['Date','count','pos','neg']].groupby(by=['Date'], as_index=False).sum()

# 날짜 데이터 merge, na -> 0으로
merge_df = pd.merge(date_df, article_df2, left_on=['Date'], right_on=['Date'], how='left')
merge_df =merge_df.fillna(0)
merge_df = merge_df[["Date","count","pos","neg"]]
# 내보내기
article_df2.to_csv('../../../Sentiment Analysis/Data/senti_CJ제일제당_V3.csv', index=False)

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
merge_df

,Date,count,pos,neg
0,2014-01-02,0.0,0.0,0.0
1,2014-01-02,0.0,0.0,0.0
2,2014-01-02,0.0,0.0,0.0
3,2014-01-02,0.0,0.0,0.0
4,2014-01-02,0.0,0.0,0.0
...,...,...,...,...
14695,2019-12-30,5.0,3.0,1.0
14696,2019-12-30,5.0,3.0,1.0
14697,2019-12-30,5.0,3.0,1.0
14698,2019-12-30,5.0,3.0,1.0


In [10]:
article_df2

,Date,count,pos,neg
0,-1,1,0,1
1,2014-01-07,1,1,0
2,2014-01-08,1,1,0
3,2014-01-09,2,2,0
4,2014-01-10,4,0,1
...,...,...,...,...
1143,2019-12-19,3,3,0
1144,2019-12-20,2,2,0
1145,2019-12-23,3,2,0
1146,2019-12-24,2,1,0


In [8]:
article_df2
# 내보내기
article_df2.to_csv('../../../Sentiment Analysis/Data/senti_CJ제일제당_V3.csv', index=False)

In [81]:
# only 중립 or 긍정=부정 -> 0
# 긍정>부정 -> 1
# 부정>긍정 -> 0
article_df2['SA'] = 0
for x in range(len(article_df2)):
    if article_df2['pos'][x]==0 & article_df2['neg'][x]==0:
        continue    
    elif (article_df2['pos'][x]-article_df2['neg'][x])<0:
        article_df2['SA'][x] = -1
    elif (article_df2['pos'][x]-article_df2['neg'][x])>0:
        article_df2['SA'][x] = 1
    else:
        continue
article_df3 = article_df2[['Date',"Category",'SA']]        

        
# article_df와 date_df merge
merge_df = pd.merge(date_df, article_df3, left_on=['Date','Category'], right_on=['Date','Category'], how='left')

# one-hot encoding
merge_df['politics']=0
merge_df['economics']=0
merge_df['IT']=0
merge_df['finance']=0
merge_df['realEstate']=0
merge_df['international']=0
merge_df['social']=0
merge_df['life']=0
merge_df['golf']=0
merge_df['ent']=0

for i in range(len(merge_df)):
    if merge_df['Category'][i] == 2:
        merge_df['politics'][i] = 1
    elif merge_df['Category'][i] == 3:
        merge_df['economics'][i] = 1
    elif merge_df['Category'][i] == 4:
        merge_df['IT'][i] = 1
    elif merge_df['Category'][i] == 5:
        merge_df['finance'][i] = 1
    elif merge_df['Category'][i] == 6:
        merge_df['realEstate'][i] = 1
    elif merge_df['Category'][i] == 7:
        merge_df['international'][i] = 1
    elif merge_df['Category'][i] == 8:
        merge_df['social'][i] = 1
    elif merge_df['Category'][i] == 9:
        merge_df['life'][i] = 1
    elif merge_df['Category'][i] == 10:
        merge_df['golf'][i] = 1
    elif merge_df['Category'][i] == 11:
        merge_df['ent'][i] = 1
    #print(i)

del merge_df['Category']

# 카테고리 줄이기
is_economics = merge_df['economics']==1
is_IT = merge_df['IT']==1
is_finance = merge_df['finance']==1
is_social = merge_df['social']==1
is_politics = merge_df['politics']==1

senti_df = merge_df[is_economics|is_IT|is_finance|is_social|is_politics]\
                [['Date', 'SA', 'economics', 'IT', 'finance', 'social', 'politics']]

# 내보내기
senti_df.to_csv('../../../Sentiment Analysis/Data/senti_CJ제일제당_V2.csv', index=False)

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launche

### 컬럼수 줄이기: 기사수가 많은 5개의 카테고리만 유지

economics, IT, finance, social, politics만 유지

In [124]:
summary = pd.read_csv('cj_final.csv')

In [150]:
temp = summary[pd.isnull(summary['label'])==False]
temp[temp.columns[2:12]].sum().sort_values(ascending=False).head(5)

economics    624
IT           555
finance      336
social        57
politics      18
dtype: int64

### 5개 카테고리에 해당하는 행, 열 추출해서 csv파일 만들기

In [172]:
is_economics = summary['economics']==1
is_IT = summary['IT']==1
is_finance = summary['finance']==1
is_social = summary['social']==1
is_politics = summary['politics']==1

senti_df = summary[is_economics|is_IT|is_finance|is_social|is_politics]\
                [['Date', 'label', 'economics', 'IT', 'finance', 'social', 'politics']]

senti_df.to_csv('../../../Sentiment Analysis/Data/senti_CJ제일제당.csv', index=False)